# Visualisation des données

### Importation des librairies

In [ ]:
# Data
import pandas as pd

## Analyse des Data

In [ ]:
normal_metadata = pd.read_excel(
    "../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/Normal.metadata.xlsx", # noqa
    sheet_name="Sheet1"
    )
normal_metadata.head()

,FILE NAME,FORMAT,SIZE,URL
0,NORMAL-1,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
1,NORMAL-2,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
2,NORMAL-3,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
3,NORMAL-4,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
4,NORMAL-5,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...


In [14]:
%nbqa flake8 notebooks/

UsageError: Line magic function `%nbqa` not found.


In [1]:
2+2

4